In [2]:
import tensorflow_hub as hub
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from python_scripts.read_xwav_header import read_xwav_header
import sys
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pickle

In [3]:
model = hub.load('https://www.kaggle.com/models/google/humpback-whale/frameworks/TensorFlow2/variations/humpback-whale/versions/1')

In [4]:
annot = pd.read_csv("data/pifsc_products_detections_annotations.csv")
annot.head()

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc
0,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1,True,True,Other,0.213,0.213,2005-06-06 13:00:00.213000+00:00,2005-06-06 13:00:00.213000+00:00
1,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,13,True,True,Other,0.108,0.108,2005-06-06 14:15:00.108000+00:00,2005-06-06 14:15:00.108000+00:00
2,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,5,True,True,Other,0.143,0.143,2005-07-02 04:00:00.143000+00:00,2005-07-02 04:00:00.143000+00:00
3,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,17,True,True,Other,0.213,0.213,2005-07-02 05:15:00.213000+00:00,2005-07-02 05:15:00.213000+00:00
4,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,22,True,True,Other,0.213,0.213,2005-08-01 10:15:00.213000+00:00,2005-08-01 10:15:00.213000+00:00


In [5]:
annot["flac_compressed_xwav_object"].nunique()

5489

In [6]:
uncompressed_files = os.listdir("data/audio_data/uncompressed")
uncompressed_files

['Cross_A_01_050606_123845.d20.x.wav',
 'Cross_A_01_050702_031345.d20.x.wav',
 'Cross_A_01_050801_074615.d20.x.wav',
 'Cross_A_01_050801_110500.d20.x.wav',
 'Cross_A_01_050908_025000.d20.x.wav',
 'Equator_A_01_120313_165615.df20.x.wav',
 'Equator_A_01_120419_012845.df20.x.wav',
 'Hawaii_K_01_070813_065730.d20.x.wav',
 'Hawaii_K_01_070813_073500.d20.x.wav',
 'Hawaii_K_01_070915_204730.d20.x.wav',
 'Hawaii_K_01_070915_212500.d20.x.wav',
 'Hawaii_K_02_080506_173715.d20.x.wav',
 'Hawaii_K_02_080506_183545.d20.x.wav',
 'Hawaii_K_02_080609_205945.d20.x.wav',
 'Hawaii_K_02_080702_115545.d20.x.wav',
 'Hawaii_K_02_080702_125715.d20.x.wav',
 'Hawaii_K_03_080807_141730.d20.x.wav',
 'Hawaii_K_03_080807_161500.d20.x.wav',
 'Hawaii_K_03_080815_114500.d20.x.wav',
 'Hawaii_K_03_080921_234730.d20.x.wav',
 'Hawaii_K_03_080922_014500.d20.x.wav',
 'Hawaii_K_03_081010_164730.d20.x.wav',
 'Hawaii_K_06_090505_164500.d20.x.wav']

In [7]:
data = {}
for file in uncompressed_files:
    if "Equator" in file:
        continue
    file_path = f"data/audio_data/uncompressed/{file}"
    data["file_name"] = file
    data["file_path"] = file_path
    data["header"] = read_xwav_header(file_path)
    
    waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(file_path))
    
    w = tf.expand_dims(waveform, 0)
    del waveform
    pcen_spec = model.front_end(w)
    del w
    data["pcen_spec"] = pcen_spec


In [9]:
with open("data/audio_data/processed/first_spcs.pkl", "wb") as f:
    pickle.dump(data, f)

In [35]:
f = "data/audio_data/uncompressed/Cross_A_01_050801_110500.d20.x.wav"
waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(f))

In [37]:
waveform

<tf.Tensor: shape=(22509000, 1), dtype=float32, numpy=
array([[0.01321411],
       [0.00869751],
       [0.00982666],
       ...,
       [0.01013184],
       [0.01025391],
       [0.01025391]], dtype=float32)>

In [39]:
read_xwav_header("data/audio_data/uncompressed/Cross_A_01_050801_110500.d20.x.wav")

({'chunk_id': b'RIFF',
  'chunk_size': 45019060,
  'format': b'WAVE',
  'subchunk1_id': b'fmt ',
  'subchunk1_size': 16,
  'audio_format': 1,
  'num_channels': 1,
  'sample_rate': 10000,
  'byte_rate': 20000,
  'block_align': 2,
  'bits_per_sample': 16},
 {'harp_subchunk_id': b'harp',
  'harp_subchunk_size': 1016,
  'wav_version_number': 1,
  'firmware_version_number': b'1.17\x00\x00\x00\x00\x00\x00',
  'instrument_id': b'DL11',
  'site_name': b'XXXX',
  'experiment_name': b'Cross_01',
  'disk_sequence_number': 9,
  'disk_serial_number': b'12345678',
  'num_of_raw_files': 30,
  'longitude': -158.25383,
  'latitude': 18.72208,
  'depth': 398,
  'reserved': b'\x00\x00\x00\x00\x00\x00\x00\x00'},
 defaultdict(dict,
             {0: {'year': 5,
               'month': 8,
               'day': 1,
               'hour': 11,
               'minute': 5,
               'second': 0,
               'ticks': 0,
               'byte_loc': 1068,
               'byte_len': 1500000,
               'wri